In [ ]:
import tkinter as tk
from tensorflow.keras.models import load_model
import pandas as pd
import pickle
import hashlib
import json
import os

# Load the pre-trained machine learning model and the scaler for data normalization
model = load_model('/Users/dev/Downloads/my_model.keras')
with open('/Users/dev/Library/Gennaker/projects/Reading/DSD/DSD_content/DSD_Qt_files/scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

# Define options for dropdown menus in the GUI and their corresponding numeric mappings for the model
yes_no_options = {'Yes': 1, 'No': 0}
sex_options = {'Male': 1, 'Female': 0}
diabetes_status = {'None': 0, 'Pre-diabetes': 1, 'Diabetes': 2}
health_levels = {f'{i + 1} - ' + level: i + 1 for i, level in enumerate(['Excellent', 'Very good', 'Good', 'Fair', 'Poor'])}
education_levels = {f'{i + 1} - ' + level: i + 1 for i, level in enumerate(['No schooling', 'Elementary', 'Some high school', 'High school graduate', 'Some college', 'College graduate'])}
income_levels = {f'{i + 1} - ' + level: i + 1 for i, level in enumerate(['< $10,000', '$10,000 - $14,999', '$15,000 - $19,999', '$20,000 - $24,999', '$25,000 - $34,999', '$35,000 - $49,999', '$50,000 - $74,999', '>= $75,000'])}
age_options = {'18-29': 1, '30-39': 2, '40-49': 3, '50-59': 4, '60-69': 5, '70+': 6}

# Define the feature order for the model which is crucial for correct model inputs during prediction
feature_order = [
    'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker', 'Stroke', 'Diabetes',
    'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare',
    'NoDocbcCost', 'GenHlth', 'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex',
    'Age', 'Education', 'Income'
]

# User data file handling for saving and loading user-specific settings and data
user_data_file = 'user_data.json'

def save_users():
    """ Save the current state of user data to a JSON file """
    with open(user_data_file, 'w') as file:
        json.dump(users, file)

def load_users():
    """ Load user data from a JSON file if it exists, otherwise return an empty dictionary """
    if os.path.exists(user_data_file):
        with open(user_data_file, 'r') as file:
            return json.load(file)
    else:
        return {}

users = load_users()

# Setup the main application GUI window
root = tk.Tk()
root.title("Health Prediction App")
var_dict = {}
current_user = {}  # A dictionary to store current user information, useful for session management

def create_option_menu(label_text, options, row):
    """ Create dropdown menus and entry fields dynamically for each question in the form """
    var = tk.StringVar(root)
    tk.Label(root, text=label_text).grid(row=row, column=0)
    if options is not None:
        menu = tk.OptionMenu(root, var, *options.keys())
        menu.grid(row=row, column=1)
        var.set(next(iter(options.keys())))  # Default to the first option
        var_dict[label_text] = (var, options)
    else:
        entry = tk.Entry(root)
        entry.grid(row=row, column=1)
        var_dict[label_text] = (entry, float)  # Assume float data for entries without dropdown options

# A list of questions and their respective options for user input
questions = [
    ('High blood pressure diagnosed?', yes_no_options),
    ('High cholesterol diagnosed?', yes_no_options),
    ('Cholesterol checked in last 5 years?', yes_no_options),
    ('Height (in inches)', None),
    ('Weight (in pounds)', None),
    ('Smoked 100+ cigarettes in life?', yes_no_options),
    ('Ever had a stroke?', yes_no_options),
    ('Diabetes status', diabetes_status),
    ('Engage in physical activity weekly?', yes_no_options),
    ('Eat fruits more than once daily?', yes_no_options),
    ('Eat vegetables more than once daily?', yes_no_options),
    ('Consume 14+ drinks of alcohol weekly?', yes_no_options),
    ('Any kind of healthcare coverage?', yes_no_options),
    ('Needed but couldn’t see doctor due to cost?', yes_no_options),
    ('General health', health_levels),
    ('Days mental health not good', None),
    ('Days physical health not good', None),
    ('Difficulty walking or climbing stairs?', yes_no_options),
    ('Sex (Male/Female)', sex_options),
    ('Age range', age_options),
    ('Highest level of education completed', education_levels),
    ('Annual household income', income_levels)
]

# Create the interface elements based on the questions
for i, (question, options) in enumerate(questions):
    create_option_menu(question, options, i)

def load_user_data(username):
    """ Load user-specific data into the form when they log in """
    user_data = users[username].get('data', {})
    for key, (var, mapping) in var_dict.items():
        if isinstance(var, tk.StringVar):
            var.set(user_data.get(key, list(mapping.keys())[0]))  # Set the default or saved data
        else:
            var.delete(0, tk.END)
            var.insert(0, user_data.get(key, ''))  # Load saved or blank for entries

def logout():
    """ Log out from the current session and show the login window """
    root.withdraw()
    show_login_window()

def predict():
    """ Collect data from the form, calculate BMI, prepare the data format for prediction, and predict the risk """
    try:
        input_data = {}
        key_mapping = {
            'High blood pressure diagnosed?': 'HighBP',
            'High cholesterol diagnosed?': 'HighChol',
            'Cholesterol checked in last 5 years?': 'CholCheck',
            'Height (in inches)': 'Height(ininches)',  # Ensure consistency in naming for model input
            'Weight (in pounds)': 'Weight(inpounds)',
            'Smoked 100+ cigarettes in life?': 'Smoker',
            'Ever had a stroke?': 'Stroke',
            'Diabetes status': 'Diabetes',
            'Engage in physical activity weekly?': 'PhysActivity',
            'Eat fruits more than once daily?': 'Fruits',
            'Eat vegetables more than once daily?': 'Veggies',
            'Consume 14+ drinks of alcohol weekly?': 'HvyAlcoholConsump',
            'Any kind of healthcare coverage?': 'AnyHealthcare',
            'Needed but couldn’t see doctor due to cost?': 'NoDocbcCost',
            'General health': 'GenHlth',
            'Days mental health not good': 'MentHlth',
            'Days physical health not good': 'PhysHlth',
            'Difficulty walking or climbing stairs?': 'DiffWalk',
            'Sex (Male/Female)': 'Sex',
            'Age range': 'Age',
            'Highest level of education completed': 'Education',
            'Annual household income': 'Income'
        }

        # Populate input_data with values from the form
        for key, (var, mapping) in var_dict.items():
            if isinstance(mapping, dict):
                input_data[key_mapping[key]] = mapping[var.get()]
            else:
                raw_value = var.get()
                input_data[key_mapping[key]] = float(raw_value) if raw_value.strip() else 0

        # Calculate BMI from height and weight if provided
        height = input_data['Height(ininches)']
        weight = input_data['Weight(inpounds)']
        bmi = (weight / (height ** 2)) * 703 if height > 0 and weight > 0 else 0
        input_data['BMI'] = bmi

        # Prepare the data for the model
        df = pd.DataFrame([input_data], columns=feature_order)
        print("DataFrame before scaling:", df)

        # Scale the data using the pre-loaded scaler
        input_scaled = scaler.transform(df)
        print("Scaled input:", input_scaled)

        # Predict using the pre-loaded model
        prediction = model.predict(input_scaled)
        percentage = prediction[0][0] * 100
        print("Prediction percentage:", percentage)

        # Update the GUI with the prediction result
        label_result.config(text=f"Prediction: {percentage:.2f}% similarity with profiles having CHD/MI")
    except Exception as e:
        label_result.config(text=f"Error: {str(e)}")
        print(f"Error: {str(e)}")

def show_login_window():
    """ Create and display the login window """
    login_window = tk.Toplevel()
    login_window.title("Login")
    tk.Label(login_window, text="Username:").pack()
    username_entry = tk.Entry(login_window)
    username_entry.pack()
    tk.Label(login_window, text="Password:").pack()
    password_entry = tk.Entry(login_window, show='*')
    password_entry.pack()

    def login():
        """ Handle user login, checking username and password """
        username = username_entry.get()
        password = hashlib.sha256(password_entry.get().encode()).hexdigest()
        if username in users and users[username]['password'] == password:
            current_user['username'] = username  # Store the current user's username
            login_window.destroy()
            load_user_data(username)
            root.deiconify()
        else:
            tk.Label(login_window, text="Invalid username or password.").pack()

    def register():
        """ Handle user registration, ensuring uniqueness of usernames """
        reg_username = username_entry.get()
        reg_password = hashlib.sha256(password_entry.get().encode()).hexdigest()
        if reg_username in users:
            tk.Label(login_window, text="Username already exists.").pack()
        else:
            users[reg_username] = {'password': reg_password, 'data': {}}
            save_users()
            tk.Label(login_window, text="Registration successful.").pack()

    tk.Button(login_window, text="Login", command=login).pack()
    tk.Button(login_window, text="Register", command=register).pack()
    tk.Button(login_window, text="Exit", command=lambda: (login_window.destroy(), root.destroy())).pack()

# Setup the main application interface
tk.Button(root, text="Predict", command=predict).grid(row=len(questions), columnspan=2)
label_result = tk.Label(root, text="Prediction will appear here", wraplength=400)  # Adjust wraplength as needed
label_result.grid(row=len(questions)+1, columnspan=2)

# Ensure proper logout handling
root.protocol("WM_DELETE_WINDOW", logout)
show_login_window()  # Show the login window on app start
root.mainloop()  # Start the main event loop of the application
